In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [5]:
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder('Downloads/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset', transform=train_transforms)
test_dataset = datasets.ImageFolder('Downloads/Pistachio_Image_Dataset/Pistachio_Image_Dataset/Pistachio_Image_Dataset', transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
class RBFNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RBFNet, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.rbf = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        x = self.hidden(x)
        x = torch.tanh(x)
        x = self.rbf(x)
        return x

In [ ]:
input_size = 64 * 64 * 3
hidden_size = 100
output_size = 2

rbf_net = RBFNet(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rbf_net.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.view(-1, input_size)
        optimizer.zero_grad()
        outputs = rbf_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    print(f'Epoch {epoch+1}/{num_epochs}, train loss: {train_loss:.4f}')